# REST API 완전 가이드

이 노트북에서는 RAG_Standard의 모든 REST API를 탐색합니다.

## 학습 내용
1. 헬스 체크 및 시스템 상태
2. 채팅 API (기본, 스트리밍, WebSocket)
3. 세션 관리
4. 피드백 및 평가
5. 관리자 API

## API 문서
- Swagger UI: http://localhost:8000/docs
- ReDoc: http://localhost:8000/redoc

In [ ]:
# 설정 및 헬퍼 함수
import json

import requests

BASE_URL = "http://localhost:8000"

# API Key 설정
# .env 파일의 FASTAPI_AUTH_KEY 값을 입력하세요
# 개발 환경에서는 비워두면 인증이 스킵됩니다
API_KEY = ""  # 예: "your-secret-key"

def pretty_print(data):
    """JSON 데이터를 보기 좋게 출력"""
    print(json.dumps(data, indent=2, ensure_ascii=False))

def api_call(method, endpoint, auth=False, **kwargs):
    """API 호출 헬퍼

    Args:
        auth: True면 X-API-Key 헤더 추가
    """
    url = f"{BASE_URL}{endpoint}"
    if auth and API_KEY:
        headers = kwargs.pop('headers', {})
        headers['X-API-Key'] = API_KEY
        kwargs['headers'] = headers
    response = requests.request(method, url, timeout=30, **kwargs)
    print(f"{method} {endpoint} -> {response.status_code}")
    return response

# 연결 확인 (공개 엔드포인트)
response = api_call("GET", "/health")
data = response.json()
print(f"✅ 서버 상태: {data['status']}")
if not API_KEY:
    print("ℹ️ API_KEY가 설정되지 않았습니다. 일부 API는 인증 오류가 발생할 수 있습니다.")

## 1. 헬스 체크 API

시스템 상태를 확인하는 엔드포인트들입니다.

In [ ]:
# GET /health - 기본 헬스 체크
response = api_call("GET", "/health")
pretty_print(response.json())

In [ ]:
# GET /stats - 시스템 리소스 통계
response = api_call("GET", "/stats")
data = response.json()

print(f"가동 시간: {data.get('uptime_human', 'N/A')}")
print(f"CPU 사용률: {data.get('cpu_percent', 0):.1f}%")
print(f"메모리 사용률: {data.get('memory_usage', {}).get('percentage', 0):.1f}%")

In [ ]:
# GET /cache-stats - 리랭킹 캐시 통계
response = api_call("GET", "/cache-stats")
data = response.json()

print(f"캐시 히트율: {data.get('hit_rate', 0):.1%}")
print(f"캐시 크기: {data.get('cache_size', 0)}/{data.get('max_size', 0)}")
print(f"절약 시간: {data.get('saved_time_ms', 0):.0f}ms")

## 2. 채팅 API

RAG 기반 채팅의 핵심 API입니다.

In [ ]:
# POST /api/chat - 기본 채팅
response = api_call("POST", "/api/chat", auth=True, json={
    "message": "RAG 시스템의 장점은?",
    "session_id": None,  # 새 세션 생성
    "stream": False,
    "use_agent": False
})

if response.status_code == 200:
    data = response.json()
    print(f"\n답변:\n{data.get('answer', '')[:500]}...")
    print("\n메타데이터:")
    print(f"  - 세션 ID: {data.get('session_id')}")
    print(f"  - 처리 시간: {data.get('processing_time', 0):.2f}초")
    print(f"  - 토큰 사용: {data.get('tokens_used', 0)}")
    # 세션 ID 저장
    SESSION_ID = data.get('session_id')
elif response.status_code == 401:
    print("❌ 인증 실패: 첫 번째 셀에서 API_KEY를 설정하세요")
    SESSION_ID = None
else:
    print(f"❌ 오류: {response.text}")
    SESSION_ID = None

In [ ]:
# GET /api/chat/stats - 채팅 통계
response = api_call("GET", "/api/chat/stats", auth=True)
if response.status_code == 200:
    pretty_print(response.json())
elif response.status_code == 401:
    print("❌ 인증 실패: API_KEY를 설정하세요")

## 3. 세션 관리

대화 세션을 관리하는 API입니다.

In [ ]:
# POST /api/chat/session - 새 세션 생성
response = api_call("POST", "/api/chat/session", auth=True, json={
    "metadata": {"purpose": "notebook_demo"}
})
if response.status_code == 200:
    new_session = response.json()
    pretty_print(new_session)
elif response.status_code == 401:
    print("❌ 인증 실패: API_KEY를 설정하세요")

In [ ]:
# GET /api/chat/session/{id}/info - 세션 상세 정보
if SESSION_ID:
    response = api_call("GET", f"/api/chat/session/{SESSION_ID}/info", auth=True)
    if response.status_code == 200:
        pretty_print(response.json())
    elif response.status_code == 401:
        print("❌ 인증 실패: API_KEY를 설정하세요")
else:
    print("먼저 채팅을 실행하여 세션을 생성하세요.")

In [ ]:
# GET /api/chat/history/{session_id} - 대화 내역 조회
if SESSION_ID:
    response = api_call("GET", f"/api/chat/history/{SESSION_ID}", auth=True, params={
        "limit": 10,
        "offset": 0
    })
    if response.status_code == 200:
        data = response.json()
        print(f"대화 내역 ({data.get('total_messages', 0)}개 메시지)\n")
        for msg in data.get('messages', []):
            role = "사용자" if msg['role'] == 'user' else "AI"
            print(f"[{role}] {msg['content'][:100]}...")
    elif response.status_code == 401:
        print("❌ 인증 실패: API_KEY를 설정하세요")
else:
    print("먼저 채팅을 실행하여 세션을 생성하세요.")

## 4. 피드백 API

답변 품질에 대한 피드백을 제출합니다.

In [ ]:
# POST /api/chat/feedback - 피드백 제출
if SESSION_ID:
    response = api_call("POST", "/api/chat/feedback", auth=True, json={
        "session_id": SESSION_ID,
        "message_id": "demo-message",
        "rating": 1,  # 1: 좋아요, -1: 싫어요
        "comment": "노트북 데모에서 테스트",
        "query": "테스트 질문",
        "response": "테스트 응답"
    })
    if response.status_code == 200:
        pretty_print(response.json())
    elif response.status_code == 401:
        print("❌ 인증 실패: API_KEY를 설정하세요")
else:
    print("먼저 채팅을 실행하여 세션을 생성하세요.")

## 5. 관리자 API

**주의:** `X-API-Key` 헤더 인증이 필요합니다.

In [ ]:
# 관리자 API 헬퍼 함수
def admin_api(method, endpoint, **kwargs):
    """관리자 API 호출 (인증 헤더 포함)

    Note: 관리자 API는 항상 X-API-Key 인증이 필요합니다.
    """
    if not API_KEY:
        print("❌ API_KEY를 먼저 설정하세요")
        return None
    headers = kwargs.pop('headers', {})
    headers['X-API-Key'] = API_KEY
    return api_call(method, endpoint, headers=headers, **kwargs)

In [ ]:
# GET /api/admin/status - 시스템 전체 상태
response = admin_api("GET", "/api/admin/status")

if response is None:
    pass  # API_KEY 미설정 시 이미 에러 출력됨
elif response.status_code == 200:
    data = response.json()
    print(f"상태: {data.get('status')}")
    print(f"가동 시간: {data.get('uptime', 0):.0f}초")
    print(f"총 문서: {data.get('total_documents', 0)}개")
    print(f"벡터 수: {data.get('vector_count', 0)}개")
elif response.status_code == 401:
    print("❌ 인증 실패: API_KEY를 확인하세요")
else:
    print(f"❌ 오류: {response.status_code}")

In [ ]:
# GET /api/admin/realtime-metrics - 실시간 모니터링
response = admin_api("GET", "/api/admin/realtime-metrics")

if response is None:
    pass  # API_KEY 미설정 시 이미 에러 출력됨
elif response.status_code == 200:
    data = response.json()
    print(f"분당 요청: {data.get('chat_requests_per_minute', 0)}")
    print(f"평균 응답시간: {data.get('average_response_time', 0):.2f}초")
    print(f"에러율: {data.get('error_rate', 0):.2%}")
elif response.status_code == 401:
    print("❌ 인증 실패: API_KEY를 확인하세요")

## 완료!

REST API의 주요 기능을 모두 탐색했습니다.

### 추가 리소스
- **Swagger UI**: http://localhost:8000/docs (모든 API 대화형 테스트)
- **[API 레퍼런스](../docs/API_REFERENCE.md)**: 전체 엔드포인트 상세 설명

### 다음 노트북
- **[03_evaluation_demo.ipynb](03_evaluation_demo.ipynb)**: 평가 시스템 탐방